In [ ]:
import copy
import os
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from scipy import stats
import pickle
from math import sqrt

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector
from tensorflow.keras.layers import Dense, Flatten, Reshape, Dropout
from tensorflow.keras.optimizers.legacy import Adam

from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [ ]:
%run utils.ipynb

In [ ]:
def predict(model, to_predict, scaler_path = '', recurrent = True):
    
    original_features = to_predict
    reconstructed_features = model.predict(to_predict)
    # If we're dealing with LSTM
    if recurrent:
        reconstructed_features = reconstructed_features.reshape(-1, to_predict.shape[2])
        original_features = original_features.reshape(-1, to_predict.shape[2])
    if scaler_path.endswith('True'):
        with open('scalers/' + scaler_path + '.pkl', 'rb') as file:
            scaler = pickle.load(file)
        original_features = scaler.inverse_transform(original_features)
        reconstructed_features = scaler.inverse_transform(reconstructed_features)
        
    return original_features, reconstructed_features

In [ ]:
def save_predict_plots(original_features, reconstructed_features, columns, num_features, name, model_type, plot = False, compute_df = True):
    
    mae_list = []
    mse_list = []
    mape_list = []
    
    # Compute df where to save metrics
    if compute_df:
        mae_df = pd.DataFrame(index = range(original_features.shape[0]), columns = columns[:num_features])
        mse_df = pd.DataFrame(index = range(original_features.shape[0]), columns = columns[:num_features])
        mape_df = pd.DataFrame(index = range(original_features.shape[0]), columns = columns[:num_features])

    for column in range(len(columns)):
        if column < num_features:
            if compute_df:
                for timestamp in range(0, original_features.shape[0]):
                    y_true = original_features[timestamp, column]
                    y_pred = reconstructed_features[timestamp, column]
                    mae = mean_absolute_error([y_true], [y_pred])
                    mse = mean_squared_error([y_true], [y_pred])
                    mape = custom_mape(np.array([y_true]), np.array([y_pred]))
                    mae_df.iloc[timestamp, column] = mae
                    mse_df.iloc[timestamp, column] = mse
                    mape_df.iloc[timestamp, column] = mape
            else:        
                mae = mean_absolute_error(original_features[column], reconstructed_features[column])
                mse = mean_squared_error(original_features[column], reconstructed_features[column])
                mape = custom_mape(original_features[column], reconstructed_features[column])
                mae_list.append(mae)
                mse_list.append(mse)
                mape_list.append(mape)
            # Plot predictions
            if plot:
                plt.figure(figsize = (10, 6))
                plt.plot(original_features[:, column], label = 'Original Features')
                #plt.plot(reconstructed_features[:, column], label = 'Reconstructed Features', linestyle = '--')
                plt.title(columns[column])
                plt.xlabel("Time Step")
                plt.ylabel("Feature Value")
                plt.legend()
                save_path = os.path.join('predictions/', f"{model_type}/{columns[column]}_{name}.png")
                plt.savefig(save_path, bbox_inches = 'tight')
                plt.close()
        
    if compute_df:
        mae_df.to_csv(os.path.join(f"predictions/{model_type}/{model_type}_mae.csv"), index = False)
        mse_df.to_csv(os.path.join(f"predictions/{model_type}/{model_type}_mse.csv"), index = False)
        mape_df.to_csv(os.path.join(f"predictions/{model_type}/{model_type}_mape.csv"), index = False)
        mae_mean = mae_df.mean(axis = 1)
        mse_mean = mse_df.mean(axis = 1)
        mape_mean = mape_df.mean(axis = 1)    
    else:
        mae_mean = np.around(sum(mae_list) / len(mae_list), 3)
        mse_mean = np.around(sum(mse_list) / len(mse_list), 3)
        mape_mean = np.around(sum(mape_list) / len(mape_list), 3)
        
    #print('Mean MAE: ' + str(mae_mean))
    #print('Mean MSE: ' + str(mse_mean))
    #print('Mean MAPE: ' + str(mape_mean))
    
    #return mae_mean
    #return mae_mean, mse_mean
    return mae_mean, mse_mean, mape_mean